V6 Update: in readme.md

# Setup and Import

In [1]:
import os
if not os.path.isfile('config.py'):
    print("config.py file not found, changing directory to parent")

config.py file not found, changing directory to parent


In [2]:
# Check for TPU availability and set it up
import os

# Check if TPU is available
global TPU_AVAILABLE
try:
    import torch_xla
    import torch_xla.core.xla_model as xm
    print("PyTorch XLA already installed")
    TPU_AVAILABLE = True
except ImportError:
    TPU_AVAILABLE = False
    print("PyTorch XLA not found, will disable TPU support")

# if no config.py file, should run the script in colab, so clone the repo
if not os.path.isfile('config.py'):
    !git clone https://github.com/LWL220184016/Balancing_Ball_for_RL.git
%cd Balancing_Ball_for_RL/
!ls

# Install necessary packages including PyTorch/XLA
!pip install -r requirements.txt

if not TPU_AVAILABLE:
    # Check what version of PyTorch we need
    import torch
    if torch.__version__.startswith('2'):
        # For PyTorch 2.x
        !pip install -q torch_xla[tpu]>=2.0
    else:
        # For PyTorch 1.x
        !pip install -q torch_xla

    # Restart runtime (required after installing PyTorch/XLA)
    print("TPU support installed. Please restart the runtime now.")
    import IPython
    IPython.display.display(IPython.display.HTML(
        "<script>google.colab.kernel.invokeFunction('notebook.Runtime.restartRuntime', [], {})</script>"
    ))
else:
    # Initialize TPU if available
    import torch_xla.core.xla_model as xm
    device = xm.xla_device()
    print(f"XLA device detected: {device}")




/usr/local/lib/python3.12/dist-packages/torch_xla/__init__.py:258: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


PyTorch XLA already installed
fatal: destination path 'Balancing_Ball_for_RL' already exists and is not an empty directory.
/content/Balancing_Ball_for_RL
'=2.0'		      game	     models	 requirements.txt
 capture	      game_history   note.txt	 RL
 check_torch_gpu.py   logs	     readme.md	 test
XLA device detected: xla:0


/tmp/ipython-input-19969390.py:43: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gymnasium as gym
import sys
import optuna

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy, ActorCriticCnnPolicy  # MLP policy instead of CNN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy

# Go up one level from the current notebook's directory to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now import from the project root
from game.balancing_ball_game import BalancingBallGame
from game.gym_env import BalancingBallEnv

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
!ls /content/

Balancing_Ball_for_RL  sample_data


In [6]:
!rm -r /content/capture
!rm -r /content/game_history
!rm -r /content/logs

rm: cannot remove '/content/capture': No such file or directory
rm: cannot remove '/content/game_history': No such file or directory
rm: cannot remove '/content/logs': No such file or directory


# Training

In [ ]:
n_envs = 5

# Choose whether to do hyperparameter optimization or direct training
do_optimization = False
from RL.levels.level3.model1.config import model_config, train_config

model_cfg = model_config()
train_cfg = train_config()

if do_optimization: # game_screen, state_based
    # from RL.optuna import Optuna_optimize
    
    # optuna_optimizer = Optuna_optimize(obs_type=model_cfg.model_obs_type, level=1)
    # n_trials = 10
    # best_trial = optuna_optimizer.optuna_parameter_tuning(n_trials=n_trials)
    # print(f"best_trial found: {best_trial}")

    pass
else:
    # Create trainer for adversarial training
    from RL.train import Train
    training = Train(
        model_cfg=model_cfg,
        train_cfg=train_cfg,
        n_envs=n_envs,
        load_model=None,  # Start fresh for adversarial training
    )

    model = training.train_ppo()

    print("Adversarial training completed!")

Initializing BalancingBallEnv...
Loading the json memory file
Loading default level configurations...
Using collision_type: {'player': 1000, 'platform': 2000, 'fallingRock': 3000}
Using player_configs: [{'shape_type': 'circle', 'size': [0.03], 'shape_mass': 1, 'shape_friction': 100, 'shape_elasticity': 0.8, 'default_position': [0.5, 0.6], 'default_velocity': [0, 0], 'abilities': ['Collision'], 'health': 10, 'color': [255, 213, 79]}]
Using level_configs: {'platform_configs': [{'shape_type': 'rectangle', 'size': [0.8, 0.04], 'shape_mass': 1, 'shape_friction': 0.7, 'shape_elasticity': 0.1, 'default_position': [0.5, 0.67], 'default_velocity': [0, 0], 'abilities': [], 'health': 'infinite', 'color': [235, 64, 52]}], 'falling_rock_configs': [{'shape_type': 'rectangle', 'size': [0.05, 0.05], 'shape_mass': 0.5, 'shape_friction': 0.7, 'shape_elasticity': 0.1, 'default_position': ['random', -0.001], 'default_velocity': ['random', 0], 'abilities': [], 'health': 'infinite', 'color': [100, 100, 100]

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Final Scores:  [147.07107117191993]
Game Over - Player fell
Final Scores:  [-40.42134242903214]
Game Over - Player fell
Final Scores:  [-81.06919681506865]
Game Over - Player fell
Final Scores:  [25.35009945779073]
Game Over - Player fell
Final Scores:  [174.02512994608017]
Game Over - Player fell
Eval num_timesteps=10000, episode_reward=42.49 +/- 100.64
Episode length: 4239.60 +/- 1024.07
---------------------------------
| eval/              |          |
|    mean_ep_length  | 4.24e+03 |
|    mean_reward     | 42.5     |
| time/              |          |
|    total_timesteps | 10000    |
---------------------------------
New best mean reward!
Final Scores:  [-44.09297424836688]
Game Over - Player fell
Final Scores:  [166.04669320968804]
Game Over - Player fell
Final Scores:  [31.175582552404617]
Game Over - Player fell
Final Scores:  [-68.26152976381486]
Game Over - Player fell
Final Scores:  [130.78774392097873]
Game Over - Player fell
Final Scores:  [99.58956003623643]
Game Over - 

In [ ]:
# Copy the best model to a stable location
!cp /content/models/best_model.zip /content/drive/MyDrive/RL_Models/best_model_$(date +%Y%m%d_%H%M%S).zip

# Optional: Monitor TPU usage
if TPU_AVAILABLE:
    !sudo lsof -w /dev/accel0

cp: cannot stat '/content/models/best_model.zip': No such file or directory
lsof: status error on /dev/accel0: No such file or directory
lsof 4.93.2
 latest revision: https://github.com/lsof-org/lsof
 latest FAQ: https://github.com/lsof-org/lsof/blob/master/00FAQ
 latest (non-formatted) man page: https://github.com/lsof-org/lsof/blob/master/Lsof.8
 usage: [-?abhKlnNoOPRtUvVX] [+|-c c] [+|-d s] [+D D] [+|-E] [+|-e s] [+|-f[gG]]
 [-F [f]] [-g [s]] [-i [i]] [+|-L [l]] [+m [m]] [+|-M] [-o [o]] [-p s]
 [+|-r [t]] [-s [p:s]] [-S [t]] [-T [t]] [-u s] [+|-w] [-x [fl]] [--] [names]
Use the ``-h'' option to get more help information.


In [ ]:
# Load a saved model and continue training or evaluate
model_path = "/content/models/best_model.zip"

if os.path.exists(model_path):
    print(f"Loading model from {model_path} for evaluation")

    # Create trainer with the saved model
    eval_trainer = Train(
        model_cfg=model_config(),
        train_cfg=train_config(),
        n_envs=1,  # Use 1 env for evaluation
    )

    # Evaluate the model
    eval_trainer.evaluate(
        model_path=model_path,
        n_episodes=5,
    )
else:
    print(f"Model not found at {model_path}")

Model not found at /content/models/best_model.zip


# --

In [ ]:
# Example of creating the environment with continuous action space for adversarial training
env = BalancingBallEnv(
    render_mode=train_cfg.render_mode,
    fps=model_cfg.fps,
    obs_type=model_cfg.model_obs_type,
    image_size=model_cfg.image_size,
    level=model_cfg.level,  # Level 3 for adversarial training
)

# Reset environment to get initial observation
obs, info = env.reset()

# Print observation and action space info
print(f"Observation shape: {obs.shape}")  # Should be (84, 84, 3) for grayscale with 3 stacked frames
print(f"Action space: {env.action_space}")  # Should be Box(low=-1, high=1, shape=(2,))
print(f"Action space shape: {env.action_space.shape}")  # Should be (2,) for two players

# Test a random continuous action
action = env.action_space.sample()
print(f"Sample action: {action}")  # Should be array of 2 values between -1 and 1

# Take a step
obs, reward, terminated, truncated, info = env.step(action)
print(f"Step result - Reward: {reward}, Individual rewards: {info.get('rewards', [])}")

# Display a sample observation (first frame only)
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4))
plt.imshow(obs[:,:,0], cmap='gray')
plt.title("Adversarial Training - Grayscale Observation")
plt.axis('off')
plt.show()

env.close()